In [ ]:
! pip install langchain==0.1.2 openai

In [1]:
# 加载环境
from dotenv import load_dotenv

load_dotenv()

True

## PromptTemplate

In [15]:
from langchain.prompts import PromptTemplate

# template = 'Tell me a {adjective} joke about {content}.'
# prompt_template = PromptTemplate.from_template(template)

# 以上两步可以合并
prompt_template = PromptTemplate.from_template(
    'Tell me a {adjective} joke about {content}.'
)

prompt = prompt_template.format(adjective='funny',content='chickens')

print(f'prompt_template: {prompt_template}')
print(f'prompt: {prompt}')



prompt_template: input_variables=['adjective', 'content'] template='Tell me a {adjective} joke about {content}.'
prompt: Tell me a funny joke about chickens.


In [11]:
prompt = PromptTemplate(
    input_variables=["adjecvtive", "content"], 
    template="Tell me a {adjective} joke about {content}."
)
prompt = prompt.format(adjective="funny", content="chickens")
print(prompt)

Tell me a funny joke about chickes.


In [3]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

## ChatPromptTemplate

In [23]:
from langchain.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)

system_template = "You are a helpful AI bot. Your name is {name}."
system_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template1 = "Hello, how are you doing?"
human_prompt1 = HumanMessagePromptTemplate.from_template(human_template1)

ai_template = "I'm doing well, thanks!"
ai_prompt = AIMessagePromptTemplate.from_template(ai_template)

human_template2 = "{user_input}"
human_prompt2 = HumanMessagePromptTemplate.from_template(human_template2)

chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt1, ai_prompt, human_prompt2])
messages = chat_prompt.format_messages(
    name='Bob',
    user_input="What's your name?"
)

print(messages)



[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content="What's your name?")]


In [18]:
from langchain.prompts import  ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name='Bob', user_input='What is your name?')
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is your name?')]


In [32]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=("You are a helpful assistant that re-writes the user's text to sound more upbeat.")
        ),
        HumanMessagePromptTemplate.from_template('{text}')
    ]
)

messages = chat_template.format_messages(text="I don't like eating tasty things.")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content="I don't like eating tasty things.")]


### LCEL

In [25]:
prompt_val = prompt_template.invoke({'adjective':'funnny', 'content':'chickens'})
prompt_val

StringPromptValue(text='Tell me a funnny joke about chickens.')

In [26]:
prompt_val.to_string()

'Tell me a funnny joke about chickens.'

In [27]:
prompt_val.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'prompts', 'base', 'StringPromptValue'],
 'kwargs': {'text': 'Tell me a funnny joke about chickens.'}}

In [28]:
prompt_val.to_messages()

[HumanMessage(content='Tell me a funnny joke about chickens.')]

In [33]:
chat_val = chat_template.invoke({"text": "i dont like eating tasty things."})
chat_val

ChatPromptValue(messages=[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content='i dont like eating tasty things.')])

In [34]:
chat_val.to_messages()

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."),
 HumanMessage(content='i dont like eating tasty things.')]

In [35]:
chat_val.to_string()

"System: You are a helpful assistant that re-writes the user's text to sound more upbeat.\nHuman: i dont like eating tasty things."

## Example selectors

### 创建样本

In [55]:
examples = [
    {"input": "hi", "output": "ciao"},
    {"input": "bye", "output": "arrivaderci"},
    {"input": "soccer", "output": "calcio"},
]

### 自定义样本选择器

In [59]:
from langchain_core.example_selectors.base import BaseExampleSelector

class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        new_world = input_variables['input']
        new_world_length = len(new_world)

        best_match = None
        smallest_diff = float('inf')

        for example in self.examples:
            current_diff = abs(len(example['input']) - new_world_length)

            if current_diff < smallest_diff:
                smallest_diff = current_diff
                best_match = example

        return [best_match]

        

In [70]:
example_selector = CustomExampleSelector(examples)

In [39]:
example_selector.select_examples({"input": "okay"})

[{'input': 'bye', 'output': 'arrivaderci'}]

In [40]:
example_selector.add_example({"input": "hand", "output": "mano"})

In [41]:
example_selector.select_examples({"input": "okay"})

[{'input': 'hand', 'output': 'mano'}]

#### 在Prompt中使用

In [74]:
from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("Input: {input} -> Output: {output}")

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Input: {input} -> Output:",
    prefix="Translate the following words from English to Italain:",
    input_variables=['input']
)

print(prompt.format(input='word'))

Translate the following words from English to Italain:

Input: bye -> Output: arrivaderci

Input: word -> Output:


### 使用基于长度的样本选择器

In [73]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector


example_prompt = PromptTemplate.from_template("Input: {input} \nOutput: {output}")

length_example_selector = LengthBasedExampleSelector(
    # 可选的样本数据
    examples=examples, 
    # 提示词模版
    example_prompt=example_prompt, 
    # 格式化的样本数据的最大长度，通过get_text_length函数来衡量
    max_length=25
)

prompt = FewShotPromptTemplate(
    example_selector=length_example_selector,
    example_prompt=example_prompt,
    suffix="Input: {input} \nOutput:",
    prefix="Translate the following words from English to Italain:",
    input_variables=['input']
)

# 输入量极小，因此所有样本数据都会被选中
print(prompt.format(input='word'))


Translate the following words from English to Italain:

Input: hi 
Output: ciao

Input: bye 
Output: arrivaderci

Input: soccer 
Output: calcio

Input: word 
Output:
